### Import some libraries:


In [118]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import sklearn.model_selection
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

### Load .csv file and separate features from labels:


In [119]:
train = pd.read_csv('train.csv')  # load .csv file as a Panda Data Frame object
#print(train)

to_drop_columns = ['Type', 'Name', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Fee', 'State', 'RescuerID', 'VideoAmt', 'Description', 'PetID']

train = train.drop(to_drop_columns, axis=1)

Y = train['AdoptionSpeed']
X = train.drop(['AdoptionSpeed'], axis=1)

#print(X)
#print(Y)

(x, y) = (X.values, Y.values)  # store features and labels into Numpy arrays (~= matrices in Python)

### Scaling features into [0; 1] floating point numbers:


In [120]:
#print(x)
#print(x[1,2])

max_x0 = 0
max_x1 = 0
max_x2 = 0

for i in range(len(x)):  # we could also use numpy.amax() function
    if x[i, 0] > max_x0:
        max_x0 = x[i, 0]
        
    if x[i, 1] > max_x1:
        max_x1 = x[i, 1]
        
    if x[i, 2] > max_x2:
        max_x2 = x[i, 2]
        
#print('maximum values : ', max_x0, max_x1, max_x2)
    
x[:,0] = x[:,0] / max_x0
x[:,1] = x[:,1] / max_x1
x[:,2] = x[:,2] / max_x2

#print(x)


### Encoding labels into one-hot encoded values:


In [121]:
#print(len(y))
y_hot = np.zeros((len(y), 5))  # y_hot is a len(y)x5 matrix

for i in range(len(y)):  # we could also use sklearn.preprocessing.OneHotEncoder() function
    if y[i]==0:
        y_hot[i, 0] = 1
    elif y[i]==1:
        y_hot[i, 1] = 1
    elif y[i]==2:
        y_hot[i, 2] = 1
    elif y[i]==3:
        y_hot[i, 3] = 1
    elif y[i]==4:
        y_hot[i, 4] = 1

#print('y not encoded')
#print(y)

y = y_hot

#print('one-hot encoded y')
#print(y)

In [122]:
# dataset split - 90% 10% - you can try something else

x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x, y, test_size=0.1, random_state=0)

### Model definition:


In [138]:
model = Sequential([
    Dense(3, input_shape=(3,)),
    Activation('relu'),
    #Dropout(0.3),
    Dense(5),
    Activation('softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#model.summary()

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')

### Model training:


In [140]:
history = model.fit(x_train, y_train, epochs=150, batch_size=100, 
                    validation_data=(x_valid, y_valid), verbose=1, callbacks=[callback_early_stopping])

In [141]:
# summarize history for loss
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

### Model testing (hand-made):


In [146]:
i = 3  # pick the i+1th sample and predict the outcome

#print('x = ', x_train[i])
x_expanded = np.expand_dims(x_train[i], 0)  
#print(x_expanded)

y_i = model.predict(x_expanded)  # predict a label value
#print('Predicted y = ', y_i)
#print('Real y = ', y_train[i])  # print the true label